# BAT.jl Tutorial - Poisson Counting Experiment

In [ ]:
using BAT
using Distributions 
using IntervalSets
using ValueShapes
using Plots
using ArraysOfArrays
using StatsBase 
using LinearAlgebra

## Situation
We want to determine the properties of a radioactive singal source in the presence of background from natural sources of radioactivity.
We assume to have one signal source $S$ and only one source of background $B$.

## 1. Background only measurement
We start by using our detector without the signal source installed.
This measurement yields a number of $k_B=10$ counts.  
Using this measurement we want to gain information about the event rate of the natural radioactive background.

### Task: 
Perform a Bayesian analysis to estimate the event rate of the natural background $\lambda_b$ using a Poisson model. 

Start by defining the model likelihood using the `logpdf()` and `Poisson()` functions from the [Distrtibutions.jl](https://juliastats.github.io/Distributions.jl/latest/univariate/) package:

In [ ]:
# Number of observed background events
kb = 10

likelihood_B = let k = kb
    params -> begin
        return logpdf(Poisson(params.λb), k) # poisson log-likelihood
    end
end;

Define the Prior with help of the `NamedPrior()` function. Use a flat prior between 0 and 30:

In [ ]:
prior_B = NamedPrior(
    λb = 0..30 
    #!EX λb = #!...!#
)

Then use the likelihood and the prior to define the `PosteriorDensity()`:

In [ ]:
posterior_B = PosteriorDensity(likelihood_B, prior_B);
#!EX posterior_B = PosteriorDensity(#!...!#); 

Define the settings for the sampling. Choose `MetropolisHstings()` as the algorithm and set the number of chains and samples:

In [ ]:
algorithm = MetropolisHastings()
nchains = 8
nsamples = 10^5;
#!EX algorithm =#!...!#)
#!EX nchains = #!...!#)
#!EX nsamples = =#!...!#)

Start the sampling by using `rand()` on the `MCMCSpec()` object with the settings defined above:

In [ ]:
samples_B, sampleids_B, stats_B, chains_B = rand(MCMCSpec(algorithm, posterior_B), nsamples, nchains);

Take a look at the resulting disribution for the background rate using `plot()`:

In [ ]:
plot(posterior_B, samples_B, :λb)
plot!(prior_B, :λb, xlabel = "\$\\lambda_b\$", ylabel = "\$P(\\lambda_b)\$")

Print some statistics of the samples:

In [ ]:
println("Mode: $(stats_B.mode)")
println("Mean: $(stats_B.param_stats.mean)")
println("Covariance: $(stats_B.param_stats.cov)")
println("Standard Deviation: $(diag(sqrt(stats_B.param_stats.cov[:, :])))")

## 2. Second background only measurement
A second measurement of the natural background yields a number of $k_B=8$ counts.  
Therefore, we want to update our estimation of the background rate using this new knowledge together with the pervious results.
### Task:
Perform an analysis of the new measurement similar to the first one.   
Use the posterior distribution of the previous background measurement as a prior for this analysis.  
This can be done by using a [StatsBase histogram](http://juliastats.github.io/StatsBase.jl/latest/empirical/#Histograms-1) of the samples: `fit(Histogram, flatview(samples)[i,:], weights, nbins)`.  
The weights of the samples need to be used with the `FrequencyWeights()` function.

Define a `StatsBase` histogram containing the previous posterior distribution:

In [ ]:
posterior_hist_B1 = fit(Histogram, flatview(samples_B.params)[1, :], FrequencyWeights(samples_B.weight), nbins = 400, closed = :left);

The histogram can be used as a prior by converting it into a univariate distribution using `BAT.HistogramAsUvDistribution(histogram)`.  
Apart from that, you can proceed similarly to the first task.

Define the likelihood:

In [ ]:
kb2 = 8

likelihood_B2 = let k = kb2
    params -> begin
        return logpdf(Poisson(params.λb), k)
    end
end
#!EX likelihood_B2 = let k = kb2 = #!...!#


Define the prior (use the posterior of the previous task) and the posterior:

In [ ]:
prior_B2 = NamedPrior(
    λb = BAT.HistogramAsUvDistribution(posterior_hist_B1)
)
#!EX prior_B2 = NamedPrior( #!...!# )

posterior_B2 = PosteriorDensity(likelihood_B2, prior_B2);
#!EX posterior_B2 #!...!#

Generate samples:

In [ ]:
samples_B2, sampleids_B2, stats_B2, chains_B2 = rand(MCMCSpec(algorithm, posterior_B2), nsamples, nchains);
#!EX samples_B2, sampleids_B2, stats_B2, chains_B2 = #!...!#

Use the  `plot(samples)` and `plot!(prior)` functions to visualize the posterior of the first analysis and the updated posterior together:

In [ ]:
plot(posterior_B2, samples_B2, :λb)
plot!(prior_B2, :λb, linewidth=1.5, xlabel = "\$\\lambda_b\$", ylabel = "\$P(\\lambda_b)\$")
#!EX #!...!#

Print some statistics of the samples:

In [ ]:
println("Mode: $(stats_B2.mode)")
println("Mean: $(stats_B2.param_stats.mean)")
println("Covariance: $(stats_B2.param_stats.cov)")
println("Standard Deviation: $(diag(sqrt(stats_B2.param_stats.cov[:, :])))")

#!EX #!...!#

## 3. Signal + Background
Including the radioactive source to the experimental setup, we repeat the measurement and obtain $k_{S+B}=12$ counts.
With this measurement and our prior knowledge about the background we are able to estimate the rate of the signal $\lambda_s$.
### Task
Perform a third analysis using a poisson model with the combined singal + background rate.
Use the known information about the background from the previous tasks as a prior and choose a suitable prior for the signal.

Define the likelihood for the signal + background model:

In [ ]:
# Number of observed events
kSB = 12

likelihood_SB = let k = kSB
    params -> begin
        return logpdf(Poisson(params.λb + params.λs), k)  # poisson log-likelihood for b+s
    end
end;
#!EX likelihood_SB = let k = kSB #!...!#

Define the prior for both the signal and backgound parameters.  
Remember to use the knowledge from the previous tasks for the background. (hint: `BAT.HistogramAsUvDistribution()`)  
Also define the posterior:

In [ ]:
hist_B2 = fit(Histogram, flatview(samples_B2.params)[1, :], FrequencyWeights(samples_B2.weight), nbins = 400, closed = :left)
#!EX #!...!#
B2 = BAT.HistogramAsUvDistribution(hist_B2);
#!EX #!...!#

prior_SB = NamedPrior(
    λb = B2,
    λs = 0..30
)
#!EX #!...!#

posterior_SB = PosteriorDensity(likelihood_SB, prior_SB);
#!EX #!...!#

Generate samples for the signal + background model:

In [ ]:
samples_SB, sampleids_SB, stats_SB, chains_SB = rand(MCMCSpec(algorithm, posterior_SB), nsamples, nchains);
#!EX samples_SB, sampleids_SB, stats_SB, chains_SB = #!...!#

To visualize an overview of the results for both prameters use `plot(samples)`.   
(Hint: use the keyword `param_labels=["\\lambda_b","\\lambda_s"]` for correct axis labels)

In [ ]:
plot(samples_SB, param_labels=["\\lambda_b","\\lambda_s"])
#!EX #!...!#

Print some statistics of the samples:

In [ ]:
println("Mode: $(stats_SB.mode)")
println("Mean: $(stats_SB.param_stats.mean)")
println("Covariance: $(stats_SB.param_stats.cov)")
println("Standard Deviation: $(diag(sqrt(stats_SB.param_stats.cov[:, :])))")

#!EX #!...!#

## 4. Error propagation

Finally, we want to caluclate the cross section of the signal process.
The rate of measured events in the detector of a couting experiment can be written as 

$\frac{\mathrm d N}{\mathrm d t} = \epsilon \cdot σ \cdot L$ ,

with the Luminosity $L$ and the efficiency of the detector $\epsilon$.   
The signal cross section is therefore given as:
### $σ_S = \frac{λ_s}{\epsilon \cdot L}$  .
For this experiment we assume a luminosity $L = 1.1$  (neglecting units).

As a final result we want to obtain either a measurement or an upper limit on the signal cross section.

### Task 4 a) Known efficiency with gaussian uncertainty
The detector efficiency has been measured to be $\epsilon = 0.1 \pm 0.02$, assuming the uncertainties to follow a normal distribution.

Calculate the signal cross section $σ_S$ using the equation above.

Use the [Distrtibutions.jl](https://juliastats.github.io/Distributions.jl/latest/univariate/) package and `rand()` to obain a sample for \epsilon.  
In order to obtain unweigthed samples of $\lambda_S$ from the previos posterior, the function `sample(array, weights, n, ordered=false, replace=false)` can be used.

The function `broadcast()` or the `.`operator (e.g. `a .+ b`) might be useful for element-wise operation when handeling the samples.  

Define the luminosity and the efficiency:

In [ ]:
L = 1.1
ϵ = rand(Normal(0.1,0.02),nsamples);
#!EX L= #!...!#
#!EX ε= #!...!#

Plot the efficiency.  
Use a [StatsBase histogram](http://juliastats.github.io/StatsBase.jl/latest/empirical/#Histograms-1) to visualize the distribution.  
(Hint: The plot recipes can also be used for the `StatsBase` histograms) 

In [ ]:
hist_ϵ = fit(Histogram, ϵ, nbins=200, closed = :left)
#!EX hist_ϵ = fit(#!...!#)

plot(hist_ϵ, 1, seriestype=:smallest_intervals, xlabel="\$\\epsilon\$", ylabel="\$p(\\epsilon)\$")

Get unweighted samples for the signal rate and calculate the cross section distribution:

In [ ]:
λ_SB = sample(flatview(samples_SB.params)[2,:], FrequencyWeights(samples_SB.weight), nsamples, ordered=false,replace=false)
σS = (λ_SB)./(ϵ*L);

#!EX λ_SB = #!...!#
#!EX σS = #!...!#

Plot the distribution of the signal cross section. (Hint: use again a histogram)  
From the plot, determine the 95% upper limit on the cross section.

In [ ]:
hist_σ = fit(Histogram, σS, nbins=200,closed = :left)
plot(hist_σ, 1, seriestype=:smallest_intervals, xlim=(0,400), xlabel="\$\\sigma_s\$", ylabel="\$p(\\sigma_s)\$")

#!EX hist_σ = #!...!#
#!EX plot(#!...!#)

### Task 4 b) Binomial analysis of calibration measurement to determine efficiency 
We now want to perform the same analysis as in task 4 a) but for the case that the detector efficiency $\epsilon$ is not yet known.   

 Instead, $\epsilon$ is to be determined using a calibration measurement with a source for which the signal rate is known. Then it is possible to calculate the efficiency from the expected number of counts and the number of counts actually measured with the detector.   


For our example, the number of expected events is assumed to be $N_\text{expected} = 200$.  
The detector measures only $N_\text{measured} = 21$ events.  

Task: Implement a binomial model using the `Binomial(n,p)` function of the [Distrtibutions.jl](https://juliastats.github.io/Distributions.jl/latest/univariate/) package and determine the distribution of the detector efficiency.  
Afterwards, repeat the steps from 4 a) using the obtained distribution of the efficiency to calculate the signal cross section .

Define the binomial likelihood:

In [ ]:
n_expected = 200
n_measured = 21

likelihood_binomial = let n = n_expected, k = n_measured
    params -> begin
        return logpdf(Binomial(n, params.p), k)
    end
end
#!EX likelihood_binomial = #!...!#

Define the prior (flat) for the efficiency and define the posterior:

In [ ]:
prior_binomial = NamedPrior(
    p = 0..1
)
#!EX prior_binomial = #!...!#

posterior_binomial = PosteriorDensity(likelihood_binomial, prior_binomial);
#!EX posterior_binomial = #!...!#

Generate the samples:

In [ ]:
samples_binomial, sampleids_binomial, stats_binomial, chains_binomial = rand(MCMCSpec(algorithm, posterior_binomial), nsamples, nchains);
#!EX samples_binomial, sampleids_binomial, stats_binomial, chains_binomial = #!...!#

Plot the distribution of the efficiency:  
(What do you observe when comparing to the efficiency used in 4 a) ?)

In [ ]:
plot(samples_binomial, 1,xlabel="\$\\epsilon\$",ylabel="\$P(\\epsilon)\$")
#!EX #!...!#

Print some statistics of the samples:

In [ ]:
println("Mode: $(stats_binomial.mode)")
println("Mean: $(stats_binomial.param_stats.mean)")
println("Covariance: $(stats_binomial.param_stats.cov)")
println("Standard Deviation: $(diag(sqrt(stats_binomial.param_stats.cov[:, :])))")
#!EX #!...!#

Calculate the cross section by sampling the same number of events from the efficiency and from the signal samples:  
(Hint: proceed like in 4 a) for the samples of $\lambda$_SB)

In [ ]:
λ_SB  = sample(samples_SB.params.data[2,:], FrequencyWeights(samples_SB.weight), 60000, replace=false, ordered=false)
ϵ   = sample(samples_binomial.params.data[1,:], FrequencyWeights(samples_binomial.weight), 60000, replace=false, ordered=false)
σS  = λ_SB./(ϵ*L);

#!EX λSB  = sample(#!...!#)
#!EX ϵ   = sample(#!...!#)
#!EX σS  = #!...!#

Use a `StatsBase`histogram to visualize the cross section distribution.  
From the plot, determine the 95% upper limit on the cross section.

In [ ]:
hist_σ = fit(Histogram, σS, nbins=200, closed = :left)
plot(hist_σ, 1, seriestype=:smallest_intervals, xlabel="\$\\sigma_s\$", ylabel="\$P(\\sigma_s)\$")
#!EX #!...!#

Compare the distribution of the signal cross section to the distribution from 4 a).